In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch

In [21]:
# Load dataset
data = pd.read_csv('output1.csv')

In [23]:
data

,content,annotation_label
0,Get fucking real dude.,1
1,She is as dirty as they come and that crook ...,1
2,why did you fuck it up. I could do it all day...,1
3,Dude they dont finish enclosing the fucking s...,1
4,WTF are you talking about Men? No men thats n...,1
...,...,...
19996,I dont. But what is complaining about it goi...,0
19997,Bahah yeah i&;m totally just gonna&; get pis...,0
19998,hahahahaha >:) im evil mwahahahahahahahahaha,0
19999,What&;s something unique about Ohio? :),0


In [25]:
# Rename target column 
data.rename(columns={'annotation_label': 'label'}, inplace=True)

In [27]:
data.head()

,content,label
0,Get fucking real dude.,1
1,She is as dirty as they come and that crook ...,1
2,why did you fuck it up. I could do it all day...,1
3,Dude they dont finish enclosing the fucking s...,1
4,WTF are you talking about Men? No men thats n...,1


In [31]:
# Split into train and test sets
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(data['content'], data['label'],test_size=0.2, random_state=42, stratify=data['label'])


In [40]:
# Load HateBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("GroNLP/hateBERT")
model = AutoModelForSequenceClassification.from_pretrained("GroNLP/hateBERT", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/hateBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:
# Tokenize data
def tokenize_function(texts):
    return tokenizer(list(texts), padding=True, truncation=True, max_length=512, return_tensors="pt")

train_encodings = tokenize_function(x_train)
val_encodings = tokenize_function(x_test)

In [64]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # Return dictionary with keys 'input_ids', 'attention_mask', and 'labels'
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx]
        }

    def __len__(self):
        return len(self.labels)


In [66]:
train_dataset = Dataset(train_encodings, y_train.tolist())
val_dataset = Dataset(val_encodings, y_test.tolist())

In [68]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True
)


In [70]:

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.513200,0.392384
2,0.204500,0.476358
3,0.134600,0.562276


TrainOutput(global_step=6000, training_loss=0.28868438343781355, metrics={'train_runtime': 92758.0488, 'train_samples_per_second': 0.517, 'train_steps_per_second': 0.065, 'total_flos': 6487331958720000.0, 'train_loss': 0.28868438343781355, 'epoch': 3.0})

In [72]:
# Evaluate the model
predictions = trainer.predict(val_dataset)
predicted_labels = predictions.predictions.argmax(axis=-1)

In [76]:
# Classification report
print(classification_report(y_test, predicted_labels, target_names=["Non-Bullying", "Bullying"]))


              precision    recall  f1-score   support

Non-Bullying       0.87      0.84      0.85      2436
    Bullying       0.76      0.81      0.78      1565

    accuracy                           0.83      4001
   macro avg       0.82      0.82      0.82      4001
weighted avg       0.83      0.83      0.83      4001



In [78]:
# Save the fine-tuned model
model.save_pretrained("./fine_tuned_hatebert")
tokenizer.save_pretrained("./fine_tuned_hatebert")

('./fine_tuned_hatebert\\tokenizer_config.json',
 './fine_tuned_hatebert\\special_tokens_map.json',
 './fine_tuned_hatebert\\vocab.txt',
 './fine_tuned_hatebert\\added_tokens.json',
 './fine_tuned_hatebert\\tokenizer.json')

In [82]:
def classify_text(text):
    # Load fine-tuned HateBERT
    tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_hatebert")
    model = AutoModelForSequenceClassification.from_pretrained("./fine_tuned_hatebert")

    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Get predictions
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

    # Map prediction to label
    label = "Non-Bullying" if predicted_class == 0 else "Bullying"
    return label

# Loop for user-defined input
while True:
    user_input = input("Enter a text to classify (or type 'exit' to quit): ")
    if user_input.lower() == "exit":
        print("Exiting...")
        break

    prediction = classify_text(user_input)
    print(f"Input: {user_input}")
    print(f"Prediction: {prediction}")


Enter a text to classify (or type 'exit' to quit):  hi


Input: hi
Prediction: Non-Bullying


Enter a text to classify (or type 'exit' to quit):  exit


Exiting...
